# Фильтрация данных

In [1]:
%%capture
# пакет для конвертации .xlsx > .csv
!pip install xlsx2csv
# cкачиваем данные с гитхаба
!wget https://raw.githubusercontent.com/shitkov/toloka_filtering/main/filtering_data_src.xlsx

In [2]:
from xlsx2csv import Xlsx2csv
Xlsx2csv('/content/filtering_data_src.xlsx', outputencoding="utf-8").convert("filtering_data_src.csv")

In [3]:
path = '/content/filtering_data_src.csv'

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv(path)

In [6]:
data.head()

,transcription
0,девушка могу ли я открыть индивидуальный пенси...
1,а вот я хочу открыть индуальный пенсионный план
2,какие коэффициенты на индивидуальном пенсионно...
3,как открыть индивидуальный пенсионный план
4,в каком возрасте можно открыть индивидуальный ...


Анализ текста показал необходимоть использования спелл-чекера для исправления опечаток. Поиск релевантных вопросов решено было проводить по словосочетанию "пенсионный план", так как были варианты индивидуальный/инвестиционный пенсионный план и сокращение "ипп".

### 1. Исправление опечаток

In [7]:
# спелл-чекер
%%capture
!pip install pyspellchecker

In [8]:
from spellchecker import SpellChecker
spell = SpellChecker(language='ru')

In [9]:
# проверка наличия слов в словаре
target_words_list = ['индивидуальный', 'пенсионный', 'план' ]
spell.known(target_words_list)

{'индивидуальный', 'пенсионный', 'план'}

###2. Чистка данных

In [10]:
import re

texts_src = data['transcription']

# убрать лишние символы
texts = [re.sub('[^а-яё ]', ' ', str(t).lower()) for t in texts_src]

# убрать лишние пробелы
texts = [re.sub(r" +", " ", t) for t in texts]

# заменить 'ипп' на 'индивидуальный пенсионный план'
target = 'пенсионный план'
texts = [t.replace('ипп', 'индивидуальный' + target) for t in texts]
data['text'] = texts

###3. Лемматизация.

In [11]:
# установка лемматизатора
%%capture
!pip install pymystem3

from pymystem3 import Mystem
mstm = Mystem()

!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

In [12]:
mstm = Mystem()

In [13]:
texts_lemm = [''.join(mstm.lemmatize(t)[:-1]) for t in texts]

###4. Фильтрация данных

In [14]:
labels = [1 if t.find(target) >= 0 else 0 for t in texts_lemm]

In [15]:
data['label'] = labels

###5. Подсчет перплексии фразы
Фраза может содержать искомое словосочетание, но не иметь смысла. Для фильтрации подобных фраз можно вычислить перплексию с помощью языковой модели и отсечь нерелевантные по импирическому порогу.

In [16]:
# список кандидатов
candidates = list(set(list(data[data['label'] == 1]['text'])))

In [17]:
%%capture
!pip install transformers sentencepiece

In [18]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
path_gpt = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer_gpt = GPT2Tokenizer.from_pretrained(path_gpt)
model_gpt = GPT2LMHeadModel.from_pretrained(path_gpt).to(device)     

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

In [20]:
def get_loss(text, tokenizer, model, device):
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
    loss = model(input_ids.to(device), labels=input_ids.to(device))[0]
    return loss.cpu().detach().numpy() * len(input_ids[0])     

In [21]:
prob_list = []
for text in candidates:
    prob = get_loss(text, tokenizer_gpt, model_gpt, device)
    prob_list.append((text, prob))

In [22]:
prob_list = sorted(prob_list, key=lambda x: x[1], reverse=True)

In [23]:
prob_list[:5]

[('здравствуйте я бы хотела бы узнать о индивидуальной пенсионной план и индивидуальныйпенсионный план как это вообще куда можно обратиться по этому вопросу можно ли',
  120.3995771408081),
 ('ассистент я бы хотел возразить индивидуальную пенсионную планку подскажи как это сделать',
  95.57284927368164),
 ('алиса сколько минимум у меня денег должен положить на индивидуальныйпенсионный план',
  91.71911716461182),
 ('здравствуйте хотела бы открыть индивидуальныйпенсионный план можете пожалуйста дать мне информацию по поводу этого',
  91.39299774169922),
 ('а что открыть индивидуальный пенсионный план нужно сразу взнос сделать или я могу через какое то время это сделать',
  88.1099796295166)]

Хвост максимальных значений перплексии показывает границу отсечки в районе 95. Для более корректного подсчета перплексии можно обучить языковую модель на целевых данных.

In [24]:
# список кандидатов а удаление
bad_candidates = []
for c in prob_list:
     if c[1] > 95:
         bad_candidates.append(c[0])

In [25]:
# изменяем метки
for b in bad_candidates:
  idx = data[data['text'] == b].index
  data.loc[idx, 'label'] = 0

In [26]:
data = data.drop(columns='text')

In [27]:
# сохраняем результат
data.to_csv('filtered_texts.csv')

In [28]:
from google.colab import files
path = '/content/filtered_texts.csv'
files.download(path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>